In [385]:
import pandas as pd
import numpy as np
import gc

In [386]:
train_folder = '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/training set'
test_folder = '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/test set'
full_folder = '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/full data'

In [387]:
gc.collect()

426

## Part A. Demo Converting Text File into Numpy Array 

In [388]:
# path_file = path_folder + '/20180624paths'

In [389]:
'''
results = []
with open(path_file) as inputfile:
    for line in inputfile:
        results.append(line.strip().split('|'))
'''

"\nresults = []\nwith open(path_file) as inputfile:\n    for line in inputfile:\n        results.append(line.strip().split('|'))\n"

In [390]:
# results

In [391]:
'''
valid_results = []
for row in results:
    if ',' in row[2]:
        valid_results.append(row)
'''        

"\nvalid_results = []\nfor row in results:\n    if ',' in row[2]:\n        valid_results.append(row)\n"

In [392]:
# valid_results

As shown, among 90551 paths in file 0624, only 20706 paths got more than 1 item presented

In [393]:
'''
arr = np.array(results)
val_arr = np.array(valid_results)
arr.shape, val_arr.shape
'''

'\narr = np.array(results)\nval_arr = np.array(valid_results)\narr.shape, val_arr.shape\n'

## Part B. Process all the Path Files in Folder

In [394]:
def txt2list(txt_path):
    '''
    input:
        1. txt_path: path to the text file
    output:
        2. a list containing individual lists, each of which contains:
            userid, pathid, itemids in path
    '''
    results = []
    with open(txt_path) as inputfile:
        for line in inputfile:
            results.append(line.strip().split('|'))
            
    valid_results = []
    for row in results:
        if ',' in row[2]:
            valid_results.append(row)
            
    return valid_results

In [395]:
from os import listdir
from os.path import isfile, join
def get_paths_from_folder(folder_path):    
    return [join(folder_path, f) for f in listdir(folder_path) if isfile(join(folder_path, f))]

In [396]:
file_paths = get_paths_from_folder(train_folder)
file_paths

['/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/training set/.DS_Store',
 '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/training set/20180624paths',
 '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/training set/20180625paths',
 '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/training set/20180626paths',
 '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/training set/20180627paths']

In [397]:
all_paths = []
for txt_path in file_paths:
    if txt_path.endswith('paths'):
        all_paths = all_paths + txt2list(txt_path)
len(all_paths)    

301510

In [398]:
arr = np.array(all_paths)
arr = arr.astype(object)
arr.shape

(301510, 3)

## Part C. Sort the Paths by Lengths

In [399]:
for j in range(len(arr)):
    arr[j][0] = int(arr[j][0])
    arr[j][1] = int(arr[j][1])
    arr[j][2] = list(arr[j][2].split(','))
    for i in range(len(arr[j][2])):
        arr[j][2][i] = int(arr[j][2][i])
        
arr

array([[113182, 2, list([1134005770, 1250913213, 861143959])],
       [125639, 7, list([702409446, 1064640040])],
       [203872, 0, list([1121720343, 1121744655, 511775377])],
       ...,
       [76298054, 10,
        list([706932246, 1182092721, 962188944, 800216295])],
       [76530139, 0, list([1214700366, 1193474742])],
       [76572166, 10, list([1106958106, 660187216])]], dtype=object)

In [400]:
arr[0]

array([113182, 2, list([1134005770, 1250913213, 861143959])], dtype=object)

In [401]:
# create a list containing the session lengths

sess_len_list = []
for i in range(arr.shape[0]):
    sess_len_list.append(len(arr[i][2]))
sess_len_arr = np.array(sess_len_list)
sess_len_arr = sess_len_arr.reshape([len(sess_len_list),1])

In [402]:
arr = np.concatenate((arr,sess_len_arr),axis = 1)
arr.shape

(301510, 4)

In [403]:
arr[0],arr[1]

(array([113182, 2, list([1134005770, 1250913213, 861143959]), 3],
       dtype=object),
 array([125639, 7, list([702409446, 1064640040]), 2], dtype=object))

#### Remarks:

Here it is necessary to note that the four columns (0-3) of the numpy array is:

0: usedid as int

1: pathid as int

2: path as np array

3: len(path) as int


In [404]:
df = pd.DataFrame(arr)
df.columns = ['userid', 'sess_id', 'sess_path', 'sess_length']

In [405]:
df = df.sort_values(by = 'sess_length')
df = df.reset_index(drop = 'True')

#### Remarks:
Here a small trouble is encountered: the pathid got duplication, but if we apply drop duplicate, disaster can happen

In [406]:
## df.drop_duplicates(subset='sess_id', keep = 'first')

In [408]:
summary = df.groupby('sess_length').count()
summary = pd.DataFrame(summary.drop(columns=['sess_id','sess_path']))
summary = summary.reset_index()
summary.columns = ['sess_length', 'count']

In [410]:
slice_table = summary
slice_table['start_index'] = slice_table['count']
slice_table['end_index'] = slice_table['count']
slice_table

,sess_length,count,start_index,end_index
0,2,202142,202142,202142
1,3,50820,50820,50820
2,4,20257,20257,20257
3,5,10362,10362,10362
4,6,5402,5402,5402
5,7,3356,3356,3356
6,8,2461,2461,2461
7,9,1718,1718,1718
8,10,990,990,990
9,11,879,879,879


In [411]:
# here is the logic for create the start index + end index
for i in range(1, len(slice_table)):
    slice_table['end_index'][i] = slice_table['end_index'][i-1] + slice_table['count'][i]

for i in range(1, len(slice_table)):
    slice_table['start_index'][i] = slice_table['end_index'][i-1]

slice_table['start_index'][0] = 0

slice_table

,sess_length,count,start_index,end_index
0,2,202142,0,202142
1,3,50820,202142,252962
2,4,20257,252962,273219
3,5,10362,273219,283581
4,6,5402,283581,288983
5,7,3356,288983,292339
6,8,2461,292339,294800
7,9,1718,294800,296518
8,10,990,296518,297508
9,11,879,297508,298387


#### Planning A:
Here it we can wrap the counting of each session length into a dictionary, when the training batch is randomly generated, the prob of session length to be selected should be proportion to the session length count.

Therefore, the index slicing range for each session length should be presented.

#### Planning B:
Use zero padding + bucketing Method to deal with the data: with different length: the candidate lengths will be: (2,3,4,5,6,10,20,50), sessions with longer length will be discarded

#### Another Planning:
The FM user/item embedding can actually be input into another RNN, say if the item/user embedding DNE: create random embedding.

(Then is it possible to train user/item embedding at RNN???

#### Some Dictionaries Needed:

In the set of active items (items that are presenting in the session paths), the item would have a unique id in this active set

so we need to map: itemid (given) ---> unique id in active item set ---> FM pretrained embedding

so we need to map: userid (given) ---> unique id in active user set --->

## Part D. User/Item Index Mapping

In [412]:
df

,userid,sess_id,sess_path,sess_length
0,76572166,10,"[1106958106, 660187216]",2
1,19455413,12,"[261802281, 424198553]",2
2,19554173,11,"[1007870857, 1230706098]",2
3,20126442,49,"[946698061, 1239559178]",2
4,30874521,2,"[883326940, 1130014937]",2
5,20674243,94,"[822285641, 955584854]",2
6,30754616,1,"[806507465, 739035082]",2
7,21210736,8,"[81309207, 1096857516]",2
8,22804499,24,"[698849876, 698789219]",2
9,25962147,1,"[1134163364, 1134391521]",2


In [413]:
# return the slicings, which will be used to separate different length of sequences
arr = df.values

In [414]:
arr.shape

(301510, 4)

In [415]:
type(df['sess_path'][1][1])

int

In [416]:
df.head()

,userid,sess_id,sess_path,sess_length
0,76572166,10,"[1106958106, 660187216]",2
1,19455413,12,"[261802281, 424198553]",2
2,19554173,11,"[1007870857, 1230706098]",2
3,20126442,49,"[946698061, 1239559178]",2
4,30874521,2,"[883326940, 1130014937]",2


In [193]:
import time

In [33]:
print(time.ctime())

# now we gonna count the active users and active items
userid_list = []
itemid_list = []
for i in range(arr.shape[0]):
    userid_list.append(arr[i][0])
    itemid_list = itemid_list + arr[i][2]
    
userid_set = set(userid_list)
itemid_set = set(itemid_list)

print('Total amount of user presented before and after dropping duplication:')
print(len(userid_list), len(userid_set))
print('Total amount of item presented before and after dropping duplication:')
print(len(itemid_list), len(itemid_set))

print(time.ctime())

Fri Jul  6 12:13:55 2018
Total amount of user presented before and after dropping duplication:
301510 35880
Total amount of item presented before and after dropping duplication:
861195 206486
Fri Jul  6 13:18:45 2018


In [38]:
userid_df = pd.DataFrame([0]+list(userid_set))
itemid_df = pd.DataFrame([0]+list(itemid_set))
userid_df.columns = ['userid']
itemid_df.columns = ['itemid']
userid_df['userid'] = pd.to_numeric(userid_df['userid'])
itemid_df['itemid'] = pd.to_numeric(itemid_df['itemid'])

userid_df = userid_df.sort_values(by = 'userid').reset_index(drop=True)
itemid_df = itemid_df.sort_values(by = 'itemid').reset_index(drop=True)

In [437]:
pro_data_fold = '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Processed Data'

In [438]:
userid_df.to_csv(pro_data_fold + '/userid_map.csv')
itemid_df.to_csv(pro_data_fold + '/itemid_map.csv')

In [177]:
# below is the dictionary for mapping item/user ids in shopee db into model based id

userid_dict = dict([(userid_df['userid'][i], i) for i in range(len(userid_df))])
itemid_dict = dict([(itemid_df['itemid'][i], i) for i in range(len(itemid_df))])

In [365]:
userid_dict

{0: 0,
 10002: 1,
 10026: 2,
 10050: 3,
 10057: 4,
 10058: 5,
 10095: 6,
 10107: 7,
 10108: 8,
 10116: 9,
 10124: 10,
 10147: 11,
 10155: 12,
 10159: 13,
 10174: 14,
 10214: 15,
 10268: 16,
 10287: 17,
 10295: 18,
 10308: 19,
 10310: 20,
 10359: 21,
 10360: 22,
 10370: 23,
 10388: 24,
 10394: 25,
 10398: 26,
 10420: 27,
 10461: 28,
 10484: 29,
 10509: 30,
 10520: 31,
 10523: 32,
 10530: 33,
 10563: 34,
 11117: 35,
 11122: 36,
 11171: 37,
 11271: 38,
 11285: 39,
 11304: 40,
 11351: 41,
 11355: 42,
 11358: 43,
 11405: 44,
 11570: 45,
 11671: 46,
 11702: 47,
 11714: 48,
 11733: 49,
 11918: 50,
 12521: 51,
 12571: 52,
 12584: 53,
 12597: 54,
 12672: 55,
 12695: 56,
 12713: 57,
 12763: 58,
 12805: 59,
 12816: 60,
 13140: 61,
 13214: 62,
 13253: 63,
 13277: 64,
 13313: 65,
 13870: 66,
 14061: 67,
 14062: 68,
 14125: 69,
 14683: 70,
 14710: 71,
 14750: 72,
 14869: 73,
 14930: 74,
 14947: 75,
 14962: 76,
 15041: 77,
 15226: 78,
 15252: 79,
 15624: 80,
 15626: 81,
 15730: 82,
 15804: 83,
 15975

In [417]:
for i in range(arr.shape[0]):
    arr[i][0] = userid_dict[arr[i][0]]
    for j in range(len(arr[i][2])):
        arr[i][2][j] = itemid_dict[arr[i][2][j]]

In [418]:
arr.shape

(301510, 4)

In [419]:
# just to show the ids in the array has been well transferred
arr[290000],arr[289999],arr[301509]

(array([7108, 13,
        list([199969, 65097, 175806, 106668, 103224, 149463, 86365]), 7],
       dtype=object), array([20670, 9,
        list([88375, 47497, 123764, 112823, 53320, 152456, 18853]), 7],
       dtype=object), array([3133, 19,
        list([69798, 5283, 7870, 55844, 4470, 83905, 101160, 60811, 60409, 60312, 22780, 24359, 5334, 63942, 77251, 35613, 15634, 50157, 16532, 11540, 16527, 16510, 16490, 20170, 27257, 27290, 11537, 19389, 70893, 75410, 62562, 75006, 28918, 20172, 6788, 66157, 28549, 32005, 50222, 47180, 34801, 34823, 35898, 10781, 676, 7164, 36033, 26440, 42214, 4855, 26814, 32995, 33405, 11840, 11033, 25315, 33377, 24472, 26556, 27711, 25467, 27881, 29241, 3783, 4407, 23324, 30576, 23079, 17375, 26069, 21927, 29736, 28587, 5374, 29312, 17573, 207, 17447, 13112, 10027, 1741, 27873, 6055, 24747, 19647, 12994, 3006, 6724, 25165, 12800, 8272, 22131, 19041, 25065, 9242, 21350, 10530, 16619, 20609, 22388, 15880, 21394, 21385, 10686, 1440, 699, 1575, 20, 6161, 2685, 34

## Part D. Data Padding & Bucketing

In [420]:
Y_list = []
for i in range(arr.shape[0]):
    Y_list.append(arr[i][2][-1])
    arr[i][2] = arr[i][2][:-1]
Y_arr = np.array(Y_list)
Y_arr = Y_arr.reshape([len(Y_list),1])

arr = np.concatenate((arr,Y_arr),axis = 1)
arr.shape

(301510, 5)

In [421]:
arr

array([[35863, 10, list([144506]), 2, 60286],
       [13691, 12, list([25622]), 2, 39979],
       [13731, 11, list([114933]), 2, 184904],
       ...,
       [13594, 81,
        list([54525, 34886, 61307, 130226, 130225, 20136, 148560, 110995, 38294, 85047, 98436, 155296, 155293, 63525, 11480, 130398, 149263, 111733, 111722, 40847, 89078, 14280, 27207, 33290, 33304, 7542, 9145, 113624, 159997, 109851, 71966, 1978, 9376, 4875, 138910, 115066, 171960, 7371, 6891, 23825, 99161, 63378, 39967, 163618, 28446, 58308, 93562, 109705, 61442, 8227, 166182, 163879, 41640, 22422, 96402, 160336, 30828, 88918, 101293, 101292, 99051, 107968, 49749, 11938, 5785, 92997, 8508, 33208, 31236, 33278, 26889, 13066, 51365, 29541, 98993, 116958, 29290, 165525, 17498, 54285, 155268, 51156, 101336, 11484, 96480, 11936, 32596, 78255, 25547, 40589, 196437, 192604]),
        93, 17489],
       [13594, 92,
        list([54525, 34886, 61307, 130226, 130225, 20136, 148560, 110995, 38294, 85047, 98436, 155296, 155293, 6

In [423]:
# we would directly process the numpy array that contains our data
# the numpy array has been processed with proper mapping
# we would replace the lists with a numpy array

buckets_list = [2,3,4,5,6,10,20,50,100]
# we deduct all elements in the buckets_list by 1, to fulfill the length of path without Y item
for i in range(len(buckets_list)):
    buckets_list[i] = buckets_list[i] - 1
    

In [425]:
def pad2buckets(buckets_list, in_arr):
    '''
    Input:
        1. buckets_list: a list of ints, indicating the step lengths of data we want to generate
        2. data_arr: the half-processed data array
    Output: 
        1. pro_arr: the processed array
    '''
    data_arr = in_arr
    # cursor labels the current max length of our training data
    cursor = 0
    for i in range(data_arr.shape[0]):
        # the fourth (index 3) column is the session length of the current user path
        if len(data_arr[i][2]) > buckets_list[-1]:
            break
        # move the cursor to the correct place, 
        # by right the max_length should be bigger or equal to the length of the current row of data
        while len(data_arr[i][2]) > buckets_list[cursor]:
            cursor = cursor + 1
        
        if len(data_arr[i][2]) == buckets_list[cursor]:
            data_arr[i][2] = np.array(data_arr[i][2])
        if len(data_arr[i][2]) < buckets_list[cursor]:
            # create a temp numpy array 
            #temp_path = np.array([0 for i in range(buckets_list[cursor])])
            for j in range(buckets_list[cursor]-len(data_arr[i][2])):
                # temp_path[j] = data_arr[i][2][j]
                data_arr[i][2].append(0)
            # data_arr[i][2] = temp_path
            data_arr[i][2] = np.array(data_arr[i][2])
            data_arr[i][3] = buckets_list[cursor] + 1
    print(i)       
    # now i is at the first row we want to abandon
    return data_arr[:i]

In [426]:
pro_arr = pad2buckets(buckets_list, arr)

301509


In [427]:
pro_df = pd.DataFrame(pro_arr)
pro_df.columns = ['userid', 'pathid', 'path', 'sess_length', 'Y']

In [428]:
pro_arr


array([[35863, 10, array([144506]), 2, 60286],
       [13691, 12, array([25622]), 2, 39979],
       [13731, 11, array([114933]), 2, 184904],
       ...,
       [13594, 91,
        array([ 54525,  34886,  61307, 130226, 130225,  20136, 148560, 110995,
        38294,  85047,  98436, 155296, 155293,  63525,  11480, 130398,
       149263, 111733, 111722,  40847,  89078,  14280,  27207,  33290,
        33304,   7542,   9145, 113624, 159997, 109851,  71966,   1978,
         9376,   4875, 138910, 115066, 171960,   7371,   6891,  23825,
        99161,  63378,  39967, 163618,  28446,  58308,  93562, 109705,
        61442,   8227, 166182, 163879,  41640,  22422,  96402, 160336,
        30828,  88918, 101293, 101292,  99051, 107968,  49749,  11938,
         5785,  92997,   8508,  33208,  31236,  33278,  26889,  13066,
        51365,  29541,  98993, 116958,  29290, 165525,  17498,  54285,
       155268,  51156, 101336,  11484,  96480,  11936,  32596,  78255,
        25547,  40589, 196437, 192604, 

In [430]:
# derive the summary/slice table for the training data

summary = pro_df.groupby('sess_length').count()
summary = pd.DataFrame(summary.drop(columns=['pathid','path','Y']))
summary = summary.reset_index()
summary.columns = ['sess_length', 'count']

slice_table = summary
slice_table['start_index'] = slice_table['count']
slice_table['end_index'] = slice_table['count']
# here is the logic for create the start index + end index
for i in range(1, len(slice_table)):
    slice_table['end_index'][i] = slice_table['end_index'][i-1] + slice_table['count'][i]

for i in range(1, len(slice_table)):
    slice_table['start_index'][i] = slice_table['end_index'][i-1]

slice_table['start_index'][0] = 0

slice_table

,sess_length,count,start_index,end_index
0,2,202142,0,202142
1,3,50820,202142,252962
2,4,20257,252962,273219
3,5,10362,273219,283581
4,6,5402,283581,288983
5,10,8525,288983,297508
6,20,3174,297508,300682
7,50,727,300682,301409
8,100,100,301409,301509


### Save Essential Utilities

In [433]:
pro_data_fold = '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Processed Data'

In [435]:
np.save(pro_data_fold+'/X_train.npy', pro_arr)

## Part E. Demo Data Generator

#### List of Utilities:
1. slice_table as df (can be generated at run time)
2. arr as numpy array: the source of training data

#### Desired Output:
1. Training item numpy array with selected session length
2. Training user numpy array
3. Training label Y

all the above are supposed to come with the same batch_size 

#### Remarks:
1. The probability of each session length to be selected as training data should be proportional to the amount of session with that length presented

#### About the GRU Model Placeholders:
1. y: label place holder: shape is num_item, generated by one hot encoding 
2. itemid (mapped): shape of (batch_size, )
3. userid (mapped): shape of (batch_size, )

#### Remarks:
1. Go check the loss function

In [ ]:
def next_batch(batch_size, pro_arr):
    '''
    Input:
        1. batch_size as integer
        2. pro_arr as numpy array, with columns: 
            [0] userid as integer
            [1] pathid as integer
            [2] path as numpy array containing itemids as integers
            [3] session_length as integer = length_input + length_output = length_input + 1
            [4] label (output itemid) as integer
    Output:
        1. next_batch_userids as numpy array, with size: [batch_size, ] or [batch_size, num_time_step]
        2. next_batch_itemids as numpy array, with size: [batch_size, num_time_step]
        3. next_batch_y as numpy array, with size: [batch_size, ]
    '''
    
    
    

In [ ]:
# step 1: decide a length for the session
# we are going to take session length <= 18 
